In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
from deep_translator import GoogleTranslator
from langdetect import detect
import time

In [1]:
dict_places_to_scrap = {
    'Just Eat': {
        'url': 'https://www.trustpilot.com/review/www.alloresto.fr',
        'avg_rating': 1.4
    },
    'Hello Fresh': {
        'url': 'https://www.trustpilot.com/review/hellofresh.com',
        'avg_rating': 3.7
    },
    'Factor_': {
        'url': 'https://www.trustpilot.com/review/factor75.com',
        'avg_rating': 4.0
    },
    'Mountain Rose Herbs': {
        'url': 'https://www.trustpilot.com/review/www.mountainroseherbs.com',
        'avg_rating': 4.8
    },
    'Hale Groves': {
        'url': 'https://www.trustpilot.com/review/halegroves.com',
        'avg_rating': 4.6
    },
    'Starbucks': {
        'url': 'https://www.trustpilot.com/review/www.starbucks.com',
        'avg_rating': 2.1
    },    
    'Fresh N Lean': {
        'url': 'https://www.trustpilot.com/review/freshnlean.com',
        'avg_rating': 4.0
    },    
    'CookUnity': {
        'url': 'https://www.trustpilot.com/review/cookunity.com',
        'avg_rating': 4.0
    },    
    'Grubhub': {
        'url': 'https://www.trustpilot.com/review/www.grubhub.com',
        'avg_rating': 3.0
    }
}

In [5]:
for marque in list(dict_places_to_scrap.keys()):
    url = dict_places_to_scrap[marque]['url']+"?sort=recency"
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    try:
        last_page = soup.find('a', {'name': 'pagination-button-last'}).find('span')
        dict_places_to_scrap[marque]['pages'] = str(last_page.text)
    except Exception:
        print(marque)
        continue
    print(last_page.text)

Just Eat
2542
2394
382
723
83
230
370
745


In [6]:
dict_places_to_scrap['Just Eat']['pages']='5'

In [31]:
df = pd.DataFrame(columns = ['Brand', 'Headline', 'Review', 'Grade'])

In [40]:
for marque in list(dict_places_to_scrap.keys()):
    time.sleep(10)
    pages_stop = 50
    if pages_stop>int(dict_places_to_scrap[marque]['pages']):
        pages_stop = int(dict_places_to_scrap[marque]['pages'])
    for i in range(int(pages_stop)):
        if(i==1):
            url = dict_places_to_scrap[marque]['url']+"?sort=recency"
        else:
            url = dict_places_to_scrap[marque]['url']+"?sort=recency"+"&page="+str(i)
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        bloc_avis = soup.findAll('section', class_="styles_reviewContentwrapper__zH_9M")
        for avis in bloc_avis:
            avis_txt = avis.find('p', class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn")
            headline = avis.find('h2', class_="typography_heading-s__f7029 typography_appearance-default__AAY17")
            note_txt = avis.find('div', class_='star-rating_starRating__4rrcf').find('img')['alt']
            note = re.search(r"[1-5]", note_txt).group()
            if(avis_txt == None):
                avis_txt = ''
            else:
                avis_txt = avis_txt.text
            df = df._append({'Brand': str(marque), 'Headline' : str(headline.text),'Review': avis_txt,                                    
                                            'Grade': note},
                                            ignore_index=True)

marque: CookUnity; pages : 370
pages_stop:50
marque: Grubhub; pages : 745
pages_stop:50


In [41]:
df

,Unnamed: 0,Brand,Headline,Review,Grade
0,0.0,Just Eat,owe me money and won’t pay,Owed money for a meal half empty and cold sinc...,1
1,1.0,Just Eat,P1ss poor. Won’t attempt to use again,"Having ordered last night, I was given a deliv...",1
2,2.0,Just Eat,Compensation thieves!,"Be careful of these money stealing, no deliver...",1
3,3.0,Just Eat,The WORST customer service,The WORST customer service. Never received my ...,1
4,4.0,Just Eat,Awful customer service,Awful customer service. For past few months I'...,1
...,...,...,...,...,...
6855,NaN,Grubhub,Grubhub was great,Grubhub was great! They delivered early and t...,5
6856,NaN,Grubhub,Grubhub is the best in Manhattan.,Grubhub is the best delivery service in Manhat...,5
6857,NaN,Grubhub,What the fu*#!!!,What the fu*#!!!Crap site. Going to bed hungry...,1
6858,NaN,Grubhub,Grub hub Driver ate my order.Haven't used Grub...,Haven't used Grub hub for sometime. They sent ...,1


We want to keep reviews only in English

In [46]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

df['langue'] = df['Review'].apply(detect_language)

print(len(df.index))
df_english_reviews = df[df['langue'] == 'en']
print(len(df_english_reviews.index))

6860
6067


In [49]:
df_english_reviews = df_english_reviews.drop(['langue'], axis=1)

In [51]:
df_english_reviews.to_csv("D:/Artus/École/A5/Machine Learning for NLP/Projet 2/trustpilot_en_50_page.csv")

# Traduction

In [75]:
df_save = df.copy()

In [108]:
df_sample = df.head(500)

In [109]:
df_sample['headline_translated'] = ''

C:\Users\artus\AppData\Local\Temp\ipykernel_28712\279632813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['headline_translated'] = ''


In [110]:
for i in range(500):
    df_sample['headline_translated'].iloc[i] = GoogleTranslator(source='auto', target='en').translate(df_sample['Headline'].iloc[i])

C:\Users\artus\AppData\Local\Temp\ipykernel_28712\4254710818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['headline_translated'].iloc[i] = GoogleTranslator(source='auto', target='en').translate(df_sample['Headline'].iloc[i])
C:\Users\artus\AppData\Local\Temp\ipykernel_28712\4254710818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['headline_translated'].iloc[i] = GoogleTranslator(source='auto', target='en').translate(df_sample['Headline'].iloc[i])
C:\Users\artus\AppData\Local\Temp\ipykernel_28712\4254710818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [111]:
df_sample

,Brand,Headline,Review,Grade,headline_translated
0,aurore market,Articles endommagés lors du transport,,3,Items damaged in transit
1,aurore market,très bien,"très bien, continuons ensemble avec la fourche...",5,Alright
2,aurore market,Grand choix de produits de qualité,"Grand choix de produits de qualité, livraison ...",5,Large selection of quality products
3,aurore market,Offres sur des produits en grande…,Offres sur des produits en grande quantité int...,4,Offers on products in large quantities…
4,aurore market,Grands choix de produits de qualités…,Grands choix de produits de qualités sauf que ...,4,Large selection of quality products…
...,...,...,...,...,...
495,aurore market,Je suis une société,Je suis une société - j'ai passé commande chez...,1,I am a company
496,aurore market,"La livraison a été remise 2 fois ,pas…","La livraison a été remise 2 fois ,pas agréable...",3,"Delivery was postponed twice, not…"
497,aurore market,Réponse très réactive,,5,Very responsive response
498,aurore market,Malgré les quelques problèmes de…,"Malgré les quelques problèmes de livraison, le...",5,Despite the few problems of…
